# Init

In [1]:
import configparser, sys

import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import kaiser

sys.path.insert(0, "../")
import toolsdiadem.tools as dia
import toolsdiadem.ser as ser
from toolsdiadem.ConfigSer import ConfigSer
from toolsdiadem.Gate import Gate

In [2]:
%matplotlib qt5
%load_ext autoreload
%autoreload 2

In [17]:
cmap='jet'
configFilename = f"C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2/WR15_plaqueSupport.ini"
configRef = f"C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2/WR15_plaqueRef.ini"
conf = ConfigSer(configFilename)
confRef = ConfigSer(configRef)
conf.printConfig()
confRef.printConfig()

centerFreq = 62.5
bandWidth = 12.4
idxElevation = 20
valMin = -90
vmin = -90
vmax = -40

C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2/WR15_plaqueSupport.ini
nb_elev 41
nb_freq 2001
nb_ssb 3
el0 0.7
az0 -0.44
base_path C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2
out_path C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2/processing/wr15
gateWidth 500.0
gateCenter 6650.0
C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2/WR15_plaqueRef.ini
nb_elev 41
nb_freq 2001
nb_ssb 3
el0 0.31
az0 -0.44
base_path C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2
out_path C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2/processing/wr15
gateWidth 0.0
gateCenter 0.0


In [4]:
withPlots = 0
dir_A = f"{conf.base_path}/20200701/000_A"

In [5]:
azSlRoElPolFFPolCATR = ser.getAzSlRoElPolFFPolCATR(dir_A)

In [6]:
azSlRoElPolFFPolCATR

array([[-4.40e-01,  1.00e-02,  0.00e+00, -3.00e-01,  1.00e+07,  4.41e+01]])

In [7]:
idxEl = 3
azSlRoElPolFFPolCATR[:,idxEl]

array([-0.3])

# Fond de chambre

In [8]:
tmp = np.loadtxt(f"{conf.out_path}/data/S11_fond_de_chambre.txt", skiprows=1)
#tmp = np.loadtxt(f"{base_path}/S11_fond_de_chambre_old_without_absA_absC.txt", skiprows=1)
S11_fondDeChambre = tmp[:,1] + 1j * tmp[:,2]

In [9]:
fig, ax = plt.subplots(1,1)
plt.plot(20 * np.log10( np.abs( np.fft.ifft( S11_fondDeChambre ) ) ), label="fond de chambre")
plt.grid()
plt.legend()

# Plaque de référence

In [24]:
tmp = np.loadtxt(f"{confRef.out_path}/data/WR15_S11_plaque_ref.txt", skiprows=1)
S11_reference = tmp[:, 1] + 1j * tmp[:, 2]
freq_reference = tmp[:, 0]

# Réflectivité
ref: ser4siepel_multiband_scan_elev_Reflectivite.sce

## Initialisation des variables

In [25]:
data_S11         = np.zeros((conf.nb_elev, conf.nb_freq), dtype=complex)
data_S11_td      = np.zeros((conf.nb_elev, conf.nb_freq), dtype=complex)
data_S11_tg      = np.zeros((conf.nb_elev, conf.nb_freq), dtype=complex)
S11_minus_moy    = np.zeros((conf.nb_elev, conf.nb_freq), dtype=complex)
S11_minus_moy_td = np.zeros((conf.nb_elev, conf.nb_freq), dtype=complex)
S11_minus_moy_tg = np.zeros((conf.nb_elev, conf.nb_freq), dtype=complex)
reflectivite     = np.zeros((conf.nb_elev, conf.nb_freq), dtype=complex)
S11_moy          = np.zeros(conf.nb_freq,            dtype=complex)
S11_moy_square   = np.zeros(conf.nb_freq,            dtype=complex)

## Lecture des données et time gating

### Lecture des données

In [27]:
seq = 0
flag_gate = 1
data_dir = dir_A

for ielev in range(conf.nb_elev):
    freq_all, S11_all = ser.getFreqAndS11(data_dir, conf.nb_ssb, seq0=seq, verbose=0)
    seq = seq + conf.nb_ssb
    data_S11[ ielev, : ] = S11_all

d = np.linspace(0, 300 / (freq_all[1] - freq_all[0]), conf.nb_freq) * 0.5

### Gate design

scipy.signal.kaiser(M, beta, sym=True)

**SYM** When True (default), generates a symmetric window, for use in filter design. When False, generates a periodic window, for use in spectral analysis

### Gating

In [33]:
myGate = Gate(d, width=conf.gateWidth, center=conf.gateCenter)
gate = myGate.gateLLC

gateSize 166


## Calcul de la réflectivité

In [34]:
reflectivite = ser.getReflectivite(data_S11, S11_reference, S11_fondDeChambre, myGate.gateLLC)

In [37]:
S11_plaque_ref_tg = np.fft.fft(np.fft.ifft(S11_reference - S11_fondDeChambre) * gate)
S11_minus_fdc = data_S11 - S11_fondDeChambre
s11_minus_fdc_td = np.fft.ifft(S11_minus_fdc, axis=1)
s11_minus_fdc_tg = s11_minus_fdc_td * gate
S11_minus_fdc_tg = np.fft.fft(s11_minus_fdc_tg, axis=1)
reflectivite = S11_minus_fdc_tg / S11_plaque_ref_tg

s11_td = np.fft.ifft(data_S11, axis=1)
s11_td_plaqueRef = np.fft.ifft(S11_reference)

In [40]:
idx = 20
plt.figure()
plt.plot(d, ser.dB(s11_td[idx, :]), label="s11_td")
plt.plot(d, ser.dB(s11_td_plaqueRef), label="s11_td_plaqueRef")
plt.plot(d, ser.dB(s11_minus_fdc_td[idx, :]), label="s11_minus_fdc_td")
plt.plot(d, ser.dB(s11_minus_fdc_tg[idx, :]), label="s11_minus_fdc_tg")
plt.legend()
plt.grid()
plt.title(f"élévation {conf.elevation[idx]}°")

Text(0.5, 1.0, 'élévation 0.7°')

# Plot

## Réflectivité de l'absorbant

In [42]:
toPlot = np.zeros( (conf.nb_elev, conf.nb_freq) )
gatebis  = 0.0 * myGate.gateLLC
gatebis[ np.where( np.abs( freq_all - 62.5 ) < 12.4 ) ] = 1.0
toPlot = 20 * np.log10( 1e-12 + np.abs( reflectivite * gatebis ) )

valMin = -90
toPlot = 0.5 * ( np.abs( toPlot - valMin ) + toPlot - valMin ) + valMin
valMax = np.amax( toPlot )
valMin = np.amin( toPlot )
print( f"valMax = {valMax:.2f} valMin = {valMin:.2f}")
     
vmin = -80
vmax = -0
fig, ax = plt.subplots( 1, 1 )
im = plt.pcolormesh(conf.elevation, np.flip( freq_all ), np.rot90(toPlot), cmap=cmap)

title = f"Réflectivité absorbant\n{data_dir}"
ax.set_title(title)
ax.set_xlabel( "Elévation [°]" )
ax.set_ylabel( "Fréquence [GHz]" )
dia.addColorBar(  im, ax, aspect='auto' )

valMax = -36.66 valMin = -90.00


# Tous les absorbants

20200701 = ["000_A", "001_C", "002_E", "003_W", "004_V", "005_G"]

20200703 = ["006_L", "007_M", "008_X", "009_H", "010_I", "011_K", "012_J", "013_O", "014_N", "015_D", "016_T", "017_AB_u", "018_N_u"]

In [44]:
np.save(f"{conf.out_path}/data/freq_WR15", freq_all)
np.save(f"{conf.out_path}/data/elevation_WR15", conf.elevation)

In [47]:
idx = np.where(np.abs(freq_all - centerFreq) < bandWidth)
idxAlt = np.where(np.abs(freq_all - centerFred) >= bandWidth)

dic_stats = {}
toPlot = np.zeros((conf.nb_elev, conf.nb_freq))

gate = myGate.gateLLC
date = "20200701"

for absorbant in conf.dic_dir[date]:

    seq = 0
    flag_gate = 1
    data_dir = f"{conf.base_path}/{date}/{absorbant}"
    descriptor = f"{absorbant}_{date}"
    
    print(f"======= {absorbant} =======")

    for ielev in range(conf.nb_elev):
        freq_all, S11_all = ser.getFreqAndS11(data_dir, conf.nb_ssb, seq0=seq, verbose=0)
        seq = seq + 3
        data_S11[ ielev, : ] = S11_all
    
    S11_plaque_ref_tg = np.fft.fft(np.fft.ifft(S11_reference - S11_fondDeChambre) * gate)
    tab_freq = freq_all
    S11_minus_moy    = data_S11 - S11_fondDeChambre
    S11_minus_moy_td = np.fft.ifft(S11_minus_moy, axis=1)
    S11_minus_moy_tg = np.fft.fft(S11_minus_moy_td * gate, axis=1)

    #=============
    # REFLECTIVITE
    reflectivite = np.zeros((conf.nb_elev, conf.nb_freq), dtype=complex)
    reflectivite = S11_minus_moy_tg / S11_plaque_ref_tg
    np.save( 
        f"{conf.out_path}/data/reflectivite/{descriptor}",
        reflectivite 
    )
    
    # drop lower and higher frequencies values
    toPlot[:,idx] = 20 * np.log10(1e-12 + np.abs(reflectivite[:,idx]))
    toPlot[:,idxAlt] = -90

    fig, ax = plt.subplots( 1, 1 )
    im = ax.pcolormesh(conf.elevation, np.flip(freq_all), np.rot90(toPlot), cmap=cmap, vmin=vmin, vmax=vmax)
    title = f"Réflectivité [{absorbant}]"
    ax.set_title(title)
    ax.set_xlabel("Elévation [°]")
    ax.set_ylabel("Fréquence [GHz]")
    dia.addColorBar(im, ax, aspect='auto')
    fig.savefig(
        f"{conf.out_path}/plot/reflectivite/reflectivite_{descriptor}.png"
    )

    #======
    # STATS
    valMax = np.amax(toPlot)
    valMin = np.amin(toPlot)
    print(f"valMax = {valMax:.2f} valMin = {valMin:.2f}")

    absReflectivite   = np.abs( reflectivite )
    reflectiviteMoy   = np.average( absReflectivite[:,idx] )
    reflectiviteSigma = np.std( absReflectivite[:,idx] )

    reflectiviteMoy_dB   = 20 * np.log10( np.abs( reflectiviteMoy ) )
    reflectiviteSigma_dB = 20 * np.log10( np.abs( reflectiviteSigma ) )
    print( f"moy = {reflectiviteMoy:.6f}, sigma = {reflectiviteSigma:.6f}" )
    print( f"moy = {reflectiviteMoy_dB:.2f} [dB], sigma = {reflectiviteSigma_dB:.2f} [dB]" )
    
    #==========
    # HISTOGRAM
    fig, ax = plt.subplots(1,1)
    ax.hist((absReflectivite[:,idx]).flatten(), 50, label="histogramme (valeurs linéaires)")
    ax.grid()
    title = f"Réflectivité, histogramme [{absorbant}]"
    ax.set_title(title)
    ax.legend()
    fig.savefig(
        f"{conf.out_path}/plot/hist/hist_{descriptor}.png"
    )
    
    #=====
    # PLOT
    fig, ax = plt.subplots(1,1)
    ax.plot( freq_all, toPlot[ idxElevation, : ] )
    ax.grid()
    title = f"Réflectivité [{absorbant}]"
    ax.set_title(title)
    ax.set_xlabel("f [GHz]")
    ax.set_ylabel("réflectivité [dB]")
    fig.savefig(
        f"{conf.out_path}/plot/reflectivite_vs_f/reflectivite_vs_f_{descriptor}.png"
    )
    
    plt.close("all")
    
    dic_stats[f"{absorbant}"] = (valMax, valMin, reflectiviteMoy, reflectiviteSigma, reflectiviteMoy_dB)

======= 000_A =======
valMax = -36.66 valMin = -101.19
moy = 0.002725, sigma = 0.002929
moy = -51.29 [dB], sigma = -50.67 [dB]
======= 001_C =======
valMax = -43.86 valMin = -108.95
moy = 0.002219, sigma = 0.001232
moy = -53.08 [dB], sigma = -58.19 [dB]
======= 002_E =======
valMax = -56.33 valMin = -119.31
moy = 0.000504, sigma = 0.000271
moy = -65.95 [dB], sigma = -71.33 [dB]
======= 003_W =======
valMax = -55.71 valMin = -113.13
moy = 0.000402, sigma = 0.000271
moy = -67.91 [dB], sigma = -71.35 [dB]
======= 004_V =======
valMax = -44.54 valMin = -116.97
moy = 0.001244, sigma = 0.001184
moy = -58.10 [dB], sigma = -58.53 [dB]
======= 005_G =======
valMax = -54.10 valMin = -108.65
moy = 0.000573, sigma = 0.000329
moy = -64.84 [dB], sigma = -69.66 [dB]


In [ ]:
for absorbant in dic_stats.keys():
    stats = dic_stats[absorbant]
    print(f"{absorbant} {stats[0]:.2f} {stats[1]:.2f} {stats[2]:.6f} {stats[3]:.6f} {stats[4]:.2f}")